In [44]:
#!pip install PyPortfolioOpt
#If you are using Colab, delete the "#" before "!pip install PyPortfolioOpt" to install this package
from pypfopt import black_litterman
from pypfopt.black_litterman import BlackLittermanModel
import pandas as pd  
import numpy as np

## Import the monthly return data from Kenneth French’s website

In [45]:
url='https://raw.githubusercontent.com/Qiaoxiaomu/LSE-FM406/main/Monthly%20Return.xlsx'#Import Data from Github
data=pd.read_excel(url, header=0)
#data=pd.read_excel('Monthly Return.xlsx', header=0) # Import from Local Files
print(data.head())

     Time   Cnsmr   Manuf   HiTec   Hlth    Other
0  199001 -0.0742 -0.0553 -0.0857 -0.0737 -0.0884
1  199002  0.0132  0.0214  0.0158 -0.0206  0.0255
2  199003  0.0457  0.0172  0.0431  0.0386 -0.0031
3  199004 -0.0148 -0.0356 -0.0281  0.0029 -0.0346
4  199005  0.0988  0.0697  0.1080  0.1276  0.0906


## Convert the data to DataFrame type & Drop the first date column

In [46]:
monthly_return_df=pd.DataFrame(data)
monthly_return=pd.DataFrame.drop(monthly_return_df,'Time',axis = 1) #drop column need to set "axis=1"
print(monthly_return.head())

    Cnsmr   Manuf   HiTec   Hlth    Other
0 -0.0742 -0.0553 -0.0857 -0.0737 -0.0884
1  0.0132  0.0214  0.0158 -0.0206  0.0255
2  0.0457  0.0172  0.0431  0.0386 -0.0031
3 -0.0148 -0.0356 -0.0281  0.0029 -0.0346
4  0.0988  0.0697  0.1080  0.1276  0.0906


## Calculate the covariance matrix of the monthly return for 5 industries

In [47]:
bl_cov=monthly_return.cov()
print(bl_cov)

          Cnsmr     Manuf     HiTec     Hlth      Other
Cnsmr  0.001618  0.001288  0.001737  0.001157  0.001833
Manuf  0.001288  0.001765  0.001718  0.001023  0.001770
HiTec  0.001737  0.001718  0.004025  0.001461  0.002287
Hlth   0.001157  0.001023  0.001461  0.002003  0.001490
Other  0.001833  0.001770  0.002287  0.001490  0.002815


## Define P & Q for different questions (i.e. import different views)

In [48]:
Q_mat_a =np.array([0.006]).reshape(-1, 1)
P_mat_a =np.array(
    [
        [1, 0, 0, 0, 0]                           #Defining P & Q for 2a (View 1 - P/Q_mat_a)
    ]
)

In [49]:
Q_mat_b = np.array([0.005]).reshape(-1, 1)
P_mat_b = np.array(
    [
        [0, 0, 1, -1, 0]                           #Defining P & Q for 2b (View 2 - P/Q_mat_b)
    ]
)

In [50]:
Q_mat = np.array([0.006,0.005]).reshape(-1, 1)
P_mat = np.array(
    [
        [1, 0, 0, 0, 0],
        [0, 0, 1, -1, 0],                           #Defining P & Q for 2c (View 1 & 2 - P/Q_mat)

    ]
)

## Import the Market Cap data & Calculate Equilibrium Weights

In [51]:
mcaps=np.array([2180667.04498336,2763094.39801794,2738911.72067274,1120974.4924056,3027882.89563838])#Import Market Cap Stat for 5 industry
mkt_weights = mcaps / mcaps.sum()#Calculating Equilibrium Weights
print(mkt_weights)

[0.1843098  0.23353651 0.2314926  0.09474467 0.25591642]


In [52]:
#mcaps={'Cnsmr':2180667.04498336,'Manuf':2763094.39801794,'HiTec':2738911.72067274,'Hlth':1120974.4924056,'Other':3027882.89563838}-Alternate

## Set risk aversion parameter & risk-free rate, Calculate Π

In [53]:
delta=1 #risk_aversion
risk_free_rate=0 #risk-free rate, not needed for this question
pi_mat=delta * bl_cov.dot(mkt_weights)+risk_free_rate #Calculating Π
print(pi_mat)

Cnsmr    0.001580
Manuf    0.001597
HiTec    0.002377
Hlth     0.001361
Other    0.002142
dtype: float64


In [54]:
#Alternate way to calculate Π using built-in function
#prior = black_litterman.market_implied_prior_returns(mcaps, delta, bl_cov,risk_free_rate=0)#Default risk_free_rate is 0.02
#wsm报错？wsm？nmd!

## Feed all the Parameters into the Black-Litterman Model

In [55]:
#Model for 2a (View 1)
model_a=BlackLittermanModel(bl_cov, 
                          pi=pi_mat, 
                          absolute_views=None, 
                          Q=Q_mat_a, P=P_mat_a, 
                          omega=None, view_confidences=None, 
                          tau=0.05, 
                          risk_aversion=1)
#Model for 2b (View 2)
model_b=BlackLittermanModel(bl_cov, 
                          pi=pi_mat, 
                          absolute_views=None, 
                          Q=Q_mat_b, P=P_mat_b, 
                          omega=None, view_confidences=None, 
                          tau=0.05, 
                          risk_aversion=1)
#Model for 2c (View 1&2)
model=BlackLittermanModel(bl_cov, 
                          pi=pi_mat, 
                          absolute_views=None, 
                          Q=Q_mat, P=P_mat, 
                          omega=None, view_confidences=None, 
                          tau=0.05, 
                          risk_aversion=1)

## Output the weights for different questions

In [56]:
#delta is risk aversion parameter, defined in In[12]
weights_a=model_a.bl_weights(delta) #Calculating the portfolio weight for 2a (View 1)
weights_b=model_b.bl_weights(delta) #Calculating the portfolio weight for 2b (View 2)
weights=model.bl_weights(delta) #Calculating the portfolio weight for 2c (View 1&2)
print("The B-L Portfolio Weight for View 1 is \n" + str(weights_a))
print("The B-L Portfolio Weight for View 2 is \n" + str(weights_b))
print("The B-L Portfolio Weight for View 1 & 2 is \n" + str(weights))

The B-L Portfolio Weight for View 1 is 
OrderedDict([('Cnsmr', 0.6552760345848999), ('Manuf', 0.09869633500010665), ('HiTec', 0.09783254199101821), ('Hlth ', 0.04004064215410142), ('Other', 0.10815444626987405)])
The B-L Portfolio Weight for View 2 is 
OrderedDict([('Cnsmr', 0.18430980129335126), ('Manuf', 0.23353651380435408), ('HiTec', 0.8729592095994716), ('Hlth ', -0.5467219407857713), ('Other', 0.25591641608859433)])
The B-L Portfolio Weight for View 1 & 2 is 
OrderedDict([('Cnsmr', 0.6410616199146655), ('Manuf', 0.10276599876843374), ('HiTec', 0.3318468702001787), ('Hlth ', -0.18828859651106866), ('Other', 0.11261410762779078)])


## Save the results to csv files

In [57]:
#model_a.save_weights_to_file('weights_a.csv') #2a (View 1)
#model_b.save_weights_to_file('weights_b.csv')#2b (View 2)
#model.save_weights_to_file('weights.csv')#2c (View 1&2)

## Change views on expected returns - Q3 

In [58]:
Q_mat_new = np.array([0.008,0.006,0.007]).reshape(-1, 1)
P_mat_new = np.array(
    [
        [0, 0, 0, 0, 1],
        [1, 0, -1, 0, 0],                           #Defining new P & Q
        [0, -1, 0, 1, 0],
    ]
)

model_new=BlackLittermanModel(bl_cov, 
                          pi=pi_mat, 
                          absolute_views=None, 
                          Q=Q_mat_new, P=P_mat_new, 
                          omega=None, view_confidences=None, 
                          tau=0.05, 
                          risk_aversion=1)

weights_new=model_new.bl_weights(delta) #Calculating the portfolio weight for changed views
print("The B-L Portfolio Weight for new Views is \n" + str(weights_new))

The B-L Portfolio Weight for new Views is 
OrderedDict([('Cnsmr', 0.7999577220749936), ('Manuf', -0.838244006094127), ('HiTec', -0.6175459037398363), ('Hlth ', 0.9822604153418573), ('Other', 0.6735717724171125)])


In [59]:
#model_new.save_weights_to_file('weights_new.csv')

## Modify degree of confidence in views (tau changes from 0.05 to 0.1) - Q4

In [60]:
#Model for 2a (View 1)
model_a_mod=BlackLittermanModel(bl_cov, 
                          pi=pi_mat, 
                          absolute_views=None, 
                          Q=Q_mat_a, P=P_mat_a, 
                          omega=None, view_confidences=None, 
                          tau=0.1, 
                          risk_aversion=1)
#Model for 2b (View 2)
model_b_mod=BlackLittermanModel(bl_cov, 
                          pi=pi_mat, 
                          absolute_views=None, 
                          Q=Q_mat_b, P=P_mat_b, 
                          omega=None, view_confidences=None, 
                          tau=0.1, 
                          risk_aversion=1)
#Model for 2c (View 1&2)
model_mod=BlackLittermanModel(bl_cov, 
                          pi=pi_mat, 
                          absolute_views=None, 
                          Q=Q_mat, P=P_mat, 
                          omega=None, view_confidences=None, 
                          tau=0.1, 
                          risk_aversion=1)
#Model with new views for 3
model_new_mod=BlackLittermanModel(bl_cov, 
                          pi=pi_mat, 
                          absolute_views=None, 
                          Q=Q_mat_new, P=P_mat_new, 
                          omega=None, view_confidences=None, 
                          tau=0.1, 
                          risk_aversion=1)

weights_a_mod=model_a_mod.bl_weights(delta) #Calculating the portfolio weight for 2a (View 1)
weights_b_mod=model_b_mod.bl_weights(delta) #Calculating the portfolio weight for 2b (View 2)
weights_mod=model_mod.bl_weights(delta) #Calculating the portfolio weight for 2c (View 1&2)
weights_new_mod=model_new_mod.bl_weights(delta) #Calculating the portfolio weight for changed views
print("The B-L Portfolio Weight for View 1 with modified confidence is \n" + str(weights_a_mod))
print("The B-L Portfolio Weight for View 2 with modified confidence is \n" + str(weights_b_mod))
print("The B-L Portfolio Weight for View 1 & 2 with modified confidence is \n" + str(weights_mod))
print("The B-L Portfolio Weight for new Views with modified confidence is \n" + str(weights_new_mod))

The B-L Portfolio Weight for View 1 with modified confidence is 
OrderedDict([('Cnsmr', 0.6552760345848999), ('Manuf', 0.09869633500010665), ('HiTec', 0.09783254199101821), ('Hlth ', 0.04004064215410142), ('Other', 0.10815444626987405)])
The B-L Portfolio Weight for View 2 with modified confidence is 
OrderedDict([('Cnsmr', 0.18430980129335126), ('Manuf', 0.23353651380435408), ('HiTec', 0.8729592095994716), ('Hlth ', -0.5467219407857713), ('Other', 0.25591641608859433)])
The B-L Portfolio Weight for View 1 & 2 with modified confidence is 
OrderedDict([('Cnsmr', 0.6410616199146655), ('Manuf', 0.10276599876843374), ('HiTec', 0.3318468702001787), ('Hlth ', -0.18828859651106866), ('Other', 0.11261410762779078)])
The B-L Portfolio Weight for new Views with modified confidence is 
OrderedDict([('Cnsmr', 0.7999577220749936), ('Manuf', -0.838244006094127), ('HiTec', -0.6175459037398363), ('Hlth ', 0.9822604153418573), ('Other', 0.6735717724171125)])


In [61]:
#model_a_mod.save_weights_to_file('weights_a_mod.csv') 
#model_b_mod.save_weights_to_file('weights_b_mod.csv')
#model_mod.save_weights_to_file('weights_mod.csv')
#model_new_mod.save_weights_to_file('weights_new_mod.csv')